In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import deeplatent as dl

if torch.cuda.is_available():
    device = torch.device("cuda:0")


In [2]:
# pass in column names for each CSV
u_cols = ['user_id',  'sex','age', 'occupation', 'zip_code']
users = pd.read_csv('../data/ml-1m/users.dat', sep='::', names=u_cols, #nrows=1000,
                    encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('../data/ml-1m/ratings.dat', sep='::', names=r_cols,
                      encoding='latin-1')

# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title', 'genre', ]
movies = pd.read_csv('../data/ml-1m/movies.dat', sep='::', names=m_cols, usecols=range(3),encoding='latin-1')
movies['genre'] = movies.genre.str.split('|')


# create one merged DataFrame
movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(users, movie_ratings)
lens['sex'] = lens['sex'].apply(lambda x: 1 if x == 'F' else 0)

In [3]:
x_train ,x_test, y_train , y_test = train_test_split(lens[['user_id','movie_id']].values, lens['rating'].values, test_size=0.1, random_state=42)

In [4]:
lens.head()

,user_id,sex,age,occupation,zip_code,movie_id,title,genre,rating,unix_timestamp
0,1,1,1,10,48067,1,Toy Story (1995),"[Animation, Children's, Comedy]",5,978824268
1,1,1,1,10,48067,48,Pocahontas (1995),"[Animation, Children's, Musical, Romance]",5,978824351
2,1,1,1,10,48067,150,Apollo 13 (1995),[Drama],5,978301777
3,1,1,1,10,48067,260,Star Wars: Episode IV - A New Hope (1977),"[Action, Adventure, Fantasy, Sci-Fi]",4,978300760
4,1,1,1,10,48067,527,Schindler's List (1993),"[Drama, War]",5,978824195


In [5]:
n_u = lens['user_id'].nunique() 
n_m = lens['movie_id'].nunique() 
print(n_u, n_m)

6040 3706


In [6]:
y = torch.from_numpy(y_train).to(torch.float64)
x = torch.from_numpy(x_train).to(torch.int64)
y_test = torch.from_numpy(y_test).to(torch.float64)
x_test = torch.from_numpy(x_test).to(torch.int64)

In [7]:
model = dl.DeepLatentNN(n_users = n_u+350, n_movies = n_m +3500, n_factors = 500)

In [8]:

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SparseAdam(model.parameters(), lr=1e-2) #, momentum =.4)
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x1 = x[:,0], x2 = x[:,1]).double()
    
    # Compute and print loss
    loss = criterion(y_pred, y).double()
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [9]:
res = model(x1 = x_test[:,0], x2 = x_test[:,1]).double()

In [10]:
criterion(y_test, res).double()

tensor(8.7673, dtype=torch.float64, grad_fn=<MeanBackward0>)